In [1]:
import pandas as pd
import requests
import json
import os
from pprint import pprint
import pymongo

In [2]:
url = "https://data.cdc.gov/resource/xbxb-epbu.json"

In [3]:
response = requests.get(url)
print(response.url)

https://data.cdc.gov/resource/xbxb-epbu.json


In [4]:
data = response.json()

In [5]:
#store JSON in a dataframe
df = pd.DataFrame.from_records(data)
df.head()

,state,year,sex,age_group,race_and_hispanic_origin,deaths,population,crude_death_rate,standard_error_for_crude_rate,lower_confidence_limit_for_crude_rate,upper_confidence_limit_for_crude_rate,age_adjusted_rate,standard_error_for_age_adjusted_rate,lower_confidence_limit_for_age_adjusted_rate,upper_confidence_limit_for_age_adjusted_rate,state_crude_rate_in_range,us_crude_rate,us_age_adjusted_rate,unit
0,Alabama,1999,Both Sexes,All Ages,All Races-All Origins,169,4430143,3.8148,0.29344,3.2396,4.3899,3.8521,0.29657,3.2708,4.4334,1.8–7.1,6.0382,6.0570,"per 100,000 population"
1,Alabama,2000,Both Sexes,All Ages,All Races-All Origins,197,4447100,4.4299,0.31561,3.8112,5.0485,4.4857,0.31985,3.8588,5.1126,1.8–7.1,6.1882,6.1749,"per 100,000 population"
2,Alabama,2001,Both Sexes,All Ages,All Races-All Origins,216,4467634,4.8348,0.32896,4.1900,5.4795,4.8915,0.33329,4.2382,5.5447,1.8–7.1,6.8057,6.7922,"per 100,000 population"
3,Alabama,2002,Both Sexes,All Ages,All Races-All Origins,211,4480089,4.7097,0.32423,4.0742,5.3452,4.7619,0.32868,4.1177,5.4062,1.8–7.1,8.1766,8.1957,"per 100,000 population"
4,Alabama,2003,Both Sexes,All Ages,All Races-All Origins,197,4503491,4.3744,0.31166,3.7635,4.9852,4.4333,0.31701,3.8120,5.0547,1.8–7.1,8.8881,8.8765,"per 100,000 population"


In [6]:
#clean dataframe
opioidabuse_df = df.copy()
opioidabuse_df.head()
opioidabuse_dfcleaned = opioidabuse_df.drop_duplicates(subset=['state', 'year'], keep='last')


In [7]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [8]:
db = client.opioidabuse_db
collection = db.opioidabuse
collection.drop()

In [9]:
for index, rows in opioidabuse_dfcleaned.iterrows():
    dictionary = {
    'state': rows [0],
    'year': rows [1],
    'sex': rows [2],
    'age_group': rows [3],
    'race_and_hispanic_origin': rows [4],
    'deaths': rows [5],
    'population': rows [6],
    'crude_death_rate': rows [7],
    'standard_error_for_crude_rate': rows [8],
    'lower_confidence_limit_for_crude_rate': rows [9],
    'upper_confidence_limit_for_crude_rate': rows [10],
    'age_adjusted_rate': rows [11], 
    'standard_error_for_age_adjusted_rate': rows [12],
    'lower_confidence_limit_for_age_adjusted_rate': rows [13],
    'upper_confidence_limit_for_age_adjusted_rate': rows [14],
    'state_crude_rate_in_range': rows [15], 
    'us_crude_rate': rows [16],
    'us_age_adjusted_rate': rows [17],
    'unit': rows [18]
        
    }
    
    collection.insert_one(dictionary)

In [10]:
for result in collection.find():
    pprint(result)

{'_id': ObjectId('5f97a065239a194935cb8c8a'),
 'age_adjusted_rate': '3.8521',
 'age_group': 'All Ages',
 'crude_death_rate': '3.8148',
 'deaths': '169',
 'lower_confidence_limit_for_age_adjusted_rate': '3.2708',
 'lower_confidence_limit_for_crude_rate': '3.2396',
 'population': '4430143',
 'race_and_hispanic_origin': 'All Races-All Origins',
 'sex': 'Both Sexes',
 'standard_error_for_age_adjusted_rate': '0.29657',
 'standard_error_for_crude_rate': '0.29344',
 'state': 'Alabama',
 'state_crude_rate_in_range': '1.8–7.1',
 'unit': 'per 100,000 population',
 'upper_confidence_limit_for_age_adjusted_rate': '4.4334',
 'upper_confidence_limit_for_crude_rate': '4.3899',
 'us_age_adjusted_rate': '6.0570',
 'us_crude_rate': '6.0382',
 'year': '1999'}
{'_id': ObjectId('5f97a065239a194935cb8c8b'),
 'age_adjusted_rate': '4.4857',
 'age_group': 'All Ages',
 'crude_death_rate': '4.4299',
 'deaths': '197',
 'lower_confidence_limit_for_age_adjusted_rate': '3.8588',
 'lower_confidence_limit_for_crude_ra

 'upper_confidence_limit_for_age_adjusted_rate': '10.8874',
 'upper_confidence_limit_for_crude_rate': '11.2252',
 'us_age_adjusted_rate': '12.2966',
 'us_crude_rate': '12.4144',
 'year': '2010'}
{'_id': ObjectId('5f97a065239a194935cb8cde'),
 'age_adjusted_rate': '10.7344',
 'age_group': 'All Ages',
 'crude_death_rate': '11.0899',
 'deaths': '4180',
 'lower_confidence_limit_for_age_adjusted_rate': '10.4059',
 'lower_confidence_limit_for_crude_rate': '10.7537',
 'population': '37691912',
 'race_and_hispanic_origin': 'All Races-All Origins',
 'sex': 'Both Sexes',
 'standard_error_for_age_adjusted_rate': '0.16762',
 'standard_error_for_crude_rate': '0.17153',
 'state': 'California',
 'state_crude_rate_in_range': '10.3–12.6',
 'unit': 'per 100,000 population',
 'upper_confidence_limit_for_age_adjusted_rate': '11.0629',
 'upper_confidence_limit_for_crude_rate': '11.4261',
 'us_age_adjusted_rate': '13.1852',
 'us_crude_rate': '13.2674',
 'year': '2011'}
{'_id': ObjectId('5f97a065239a194935cb8

 'age_adjusted_rate': '16.2362',
 'age_group': 'All Ages',
 'crude_death_rate': '15.9240',
 'deaths': '3228',
 'lower_confidence_limit_for_age_adjusted_rate': '15.6608',
 'lower_confidence_limit_for_crude_rate': '15.3747',
 'population': '20271272',
 'race_and_hispanic_origin': 'All Races-All Origins',
 'sex': 'Both Sexes',
 'standard_error_for_age_adjusted_rate': '0.29356',
 'standard_error_for_crude_rate': '0.28028',
 'state': 'Florida',
 'state_crude_rate_in_range': '16–52',
 'unit': 'per 100,000 population',
 'upper_confidence_limit_for_age_adjusted_rate': '16.8116',
 'upper_confidence_limit_for_crude_rate': '16.4734',
 'us_age_adjusted_rate': '16.2923',
 'us_crude_rate': '16.3040',
 'year': '2015'}
{'_id': ObjectId('5f97a065239a194935cb8d3d'),
 'age_adjusted_rate': '23.7451',
 'age_group': 'All Ages',
 'crude_death_rate': '22.9376',
 'deaths': '4728',
 'lower_confidence_limit_for_age_adjusted_rate': '23.0498',
 'lower_confidence_limit_for_crude_rate': '22.2838',
 'population': '20

 'state_crude_rate_in_range': '16–52',
 'unit': 'per 100,000 population',
 'upper_confidence_limit_for_age_adjusted_rate': '17.6096',
 'upper_confidence_limit_for_crude_rate': '17.1656',
 'us_age_adjusted_rate': '13.8005',
 'us_crude_rate': '13.9127',
 'year': '2013'}
{'_id': ObjectId('5f97a065239a194935cb8d95'),
 'age_adjusted_rate': '18.1805',
 'age_group': 'All Ages',
 'crude_death_rate': '17.7660',
 'deaths': '1172',
 'lower_confidence_limit_for_age_adjusted_rate': '17.1218',
 'lower_confidence_limit_for_crude_rate': '16.7489',
 'population': '6596855',
 'race_and_hispanic_origin': 'All Races-All Origins',
 'sex': 'Both Sexes',
 'standard_error_for_age_adjusted_rate': '0.54018',
 'standard_error_for_crude_rate': '0.51895',
 'state': 'Indiana',
 'state_crude_rate_in_range': '16–52',
 'unit': 'per 100,000 population',
 'upper_confidence_limit_for_age_adjusted_rate': '19.2393',
 'upper_confidence_limit_for_crude_rate': '18.7832',
 'us_age_adjusted_rate': '14.6831',
 'us_crude_rate': '

 'race_and_hispanic_origin': 'All Races-All Origins',
 'sex': 'Both Sexes',
 'standard_error_for_age_adjusted_rate': '0.59338',
 'standard_error_for_crude_rate': '0.59681',
 'state': 'Maryland',
 'state_crude_rate_in_range': '16–52',
 'unit': 'per 100,000 population',
 'upper_confidence_limit_for_age_adjusted_rate': '22.0353',
 'upper_confidence_limit_for_crude_rate': '22.5636',
 'us_age_adjusted_rate': '16.2923',
 'us_crude_rate': '16.3040',
 'year': '2015'}
{'_id': ObjectId('5f97a065239a194935cb8e03'),
 'age_adjusted_rate': '33.2231',
 'age_group': 'All Ages',
 'crude_death_rate': '33.9735',
 'deaths': '2044',
 'lower_confidence_limit_for_age_adjusted_rate': '31.7533',
 'lower_confidence_limit_for_crude_rate': '32.5007',
 'population': '6016447',
 'race_and_hispanic_origin': 'All Races-All Origins',
 'sex': 'Both Sexes',
 'standard_error_for_age_adjusted_rate': '0.74992',
 'standard_error_for_crude_rate': '0.75145',
 'state': 'Maryland',
 'state_crude_rate_in_range': '16–52',
 'unit'

{'_id': ObjectId('5f97a065239a194935cb8e55'),
 'age_adjusted_rate': '13.1468',
 'age_group': 'All Ages',
 'crude_death_rate': '12.9138',
 'deaths': '765',
 'lower_confidence_limit_for_age_adjusted_rate': '12.2043',
 'lower_confidence_limit_for_crude_rate': '11.9986',
 'population': '5923916',
 'race_and_hispanic_origin': 'All Races-All Origins',
 'sex': 'Both Sexes',
 'standard_error_for_age_adjusted_rate': '0.48083',
 'standard_error_for_crude_rate': '0.46690',
 'state': 'Missouri',
 'state_crude_rate_in_range': '12.6–16',
 'unit': 'per 100,000 population',
 'upper_confidence_limit_for_age_adjusted_rate': '14.0892',
 'upper_confidence_limit_for_crude_rate': '13.8289',
 'us_age_adjusted_rate': '11.8947',
 'us_crude_rate': '11.9864',
 'year': '2008'}
{'_id': ObjectId('5f97a065239a194935cb8e56'),
 'age_adjusted_rate': '14.6489',
 'age_group': 'All Ages',
 'crude_death_rate': '14.4772',
 'deaths': '863',
 'lower_confidence_limit_for_age_adjusted_rate': '13.6583',
 'lower_confidence_limit_

 'us_crude_rate': '19.6925',
 'year': '2016'}
{'_id': ObjectId('5f97a065239a194935cb8ea6'),
 'age_adjusted_rate': '6.5027',
 'age_group': 'All Ages',
 'crude_death_rate': '6.6630',
 'deaths': '557',
 'lower_confidence_limit_for_age_adjusted_rate': '5.9611',
 'lower_confidence_limit_for_crude_rate': '6.1097',
 'population': '8359593',
 'race_and_hispanic_origin': 'All Races-All Origins',
 'sex': 'Both Sexes',
 'standard_error_for_age_adjusted_rate': '0.27628',
 'standard_error_for_crude_rate': '0.28232',
 'state': 'New Jersey',
 'state_crude_rate_in_range': '1.8–7.1',
 'unit': 'per 100,000 population',
 'upper_confidence_limit_for_age_adjusted_rate': '7.0442',
 'upper_confidence_limit_for_crude_rate': '7.2164',
 'us_age_adjusted_rate': '6.0570',
 'us_crude_rate': '6.0382',
 'year': '1999'}
{'_id': ObjectId('5f97a065239a194935cb8ea7'),
 'age_adjusted_rate': '7.5388',
 'age_group': 'All Ages',
 'crude_death_rate': '7.6417',
 'deaths': '643',
 'lower_confidence_limit_for_age_adjusted_rate'

 'lower_confidence_limit_for_age_adjusted_rate': '3.2358',
 'lower_confidence_limit_for_crude_rate': '3.3528',
 'population': '652822',
 'race_and_hispanic_origin': 'All Races-All Origins',
 'sex': 'Both Sexes',
 'standard_error_for_age_adjusted_rate': '0.85627',
 'standard_error_for_crude_rate': '0.86652',
 'state': 'North Dakota',
 'state_crude_rate_in_range': '1.8–7.1',
 'unit': 'per 100,000 population',
 'upper_confidence_limit_for_age_adjusted_rate': '6.7597',
 'upper_confidence_limit_for_crude_rate': '6.9199',
 'us_age_adjusted_rate': '11.8775',
 'us_crude_rate': '11.9543',
 'year': '2007'}
{'_id': ObjectId('5f97a065239a194935cb8ef7'),
 'age_adjusted_rate': '7.3703',
 'age_group': 'All Ages',
 'crude_death_rate': '6.9955',
 'deaths': '46',
 'lower_confidence_limit_for_age_adjusted_rate': '5.3553',
 'lower_confidence_limit_for_crude_rate': '5.1216',
 'population': '657569',
 'race_and_hispanic_origin': 'All Races-All Origins',
 'sex': 'Both Sexes',
 'standard_error_for_age_adjuste

 'unit': 'per 100,000 population',
 'upper_confidence_limit_for_age_adjusted_rate': '20.2381',
 'upper_confidence_limit_for_crude_rate': '20.6427',
 'us_age_adjusted_rate': '13.1852',
 'us_crude_rate': '13.2674',
 'year': '2011'}
{'_id': ObjectId('5f97a065239a194935cb8f55'),
 'age_adjusted_rate': '18.2384',
 'age_group': 'All Ages',
 'crude_death_rate': '18.6615',
 'deaths': '196',
 'lower_confidence_limit_for_age_adjusted_rate': '15.6202',
 'lower_confidence_limit_for_crude_rate': '16.0489',
 'population': '1050292',
 'race_and_hispanic_origin': 'All Races-All Origins',
 'sex': 'Both Sexes',
 'standard_error_for_age_adjusted_rate': '1.33584',
 'standard_error_for_crude_rate': '1.33296',
 'state': 'Rhode Island',
 'state_crude_rate_in_range': '16–52',
 'unit': 'per 100,000 population',
 'upper_confidence_limit_for_age_adjusted_rate': '20.8567',
 'upper_confidence_limit_for_crude_rate': '21.2741',
 'us_age_adjusted_rate': '13.1422',
 'us_crude_rate': '13.2208',
 'year': '2012'}
{'_id': 